In [67]:
## Run detect.py, set save_txt to True, run this then on the the output of that
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.geometry import box
import shapely.wkt
from shapely.wkt import loads
from tqdm import tqdm
import pandas as pd
import os
import sys
import numpy as np
import glob
from solaris.vector.polygon import convert_poly_coords
from solaris.vector.mask import mask_to_poly_geojson
from solaris.utils.geo import get_crs
import gdal

In [68]:
def non_max_suppression(boxes, probs=[], overlapThresh=0.5):
    """
    Apply non-max suppression.
    Directly from:
    https://github.com/CosmiQ/simrdwn/
    http://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
    Malisiewicz et al.
    see modular_sliding_window.py, functions non_max_suppression, \
            non_max_supression_rot
    Arguments
    ---------
    boxes : np.array
        Prediction boxes with the format: [[xmin, ymin, xmax, ymax], [...] ]
    probs : np.array
        Array of prediction scores or probabilities.  If [], ignore.  If not
        [], sort boxes by probability prior to applying non-max suppression.
        Defaults to ``[]``.
    overlapThresh : float
        minimum IOU overlap to retain.  Defaults to ``0.5``.
    Returns
    -------
    pick : np.array
        Array of indices to keep
    """

    print("Executing non-max suppression...")
    len_init = len(boxes)

    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return [], [], []

    # boxes_tot = boxes  # np.asarray(boxes)
    boxes = np.asarray([b[:4] for b in boxes])
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    # compute the area of the bounding boxes
    area = (x2 - x1) * (y2 - y1)

    # sort the boxes by the bottom-right y-coordinate of the bounding box
    if len(probs) == 0:
        idxs = np.argsort(((x2 - x1) * (y2 - y1)))
    # sort boxes by the highest prob (descending order)
    else:
        idxs = np.argsort(probs)[::-1]

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1)
        h = np.maximum(0, yy2 - yy1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have
        idxs = np.delete(
            idxs,
            np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    print("  non-max suppression init boxes:", len_init)
    print("  non-max suppression final boxes:", len(pick))
    return pick

def vectorize_yolo(df, output_dir, nms=True, overlapThresh=0.5):
    # Create directory
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    bb_dir = os.path.join(output_dir, "bounding_boxes")
    if not os.path.exists(bb_dir):
        os.makedirs(bb_dir)
    try:
        df['bb_geometry'] = df['bb_geometry'].apply(loads)
    except AttributeError:
        pass
    for img_id in df['image'].unique():
        img_df = df[df['image'] == img_id]
        img_df = img_df.reset_index()
        crs = str('EPSG:' + img_df['crs'].iloc[0])
        if nms is True:
            confs = []
            bbs = []
            for bb_geom in img_df['bb_geometry']:
                bb_geom = [bb_geom.bounds[0],bb_geom.bounds[1],bb_geom.bounds[2],bb_geom.bounds[3]]
                bbs.append(bb_geom)
            b_box_nms_indices = non_max_suppression(bbs, confs, overlapThresh=overlapThresh)
            b_box_df_out = gpd.GeoDataFrame(img_df.loc[b_box_nms_indices], crs=crs, geometry=img_df.bb_geometry)
            b_box_df_out = b_box_df_out.drop(columns=["bb_geometry", "crs", 'index'])
        else:
            b_box_df_out = gpd.GeoDataFrame(img_df, crs=crs, geometry=img_df.bb_geometry)
            b_box_df_out = b_box_df_out.drop(columns=["bb_geometry", "crs", 'index'])
        output = os.path.join(bb_dir, img_id.split(".")[0] + ".geojson")
        b_box_df_out.to_file(output, driver='GeoJSON')

In [96]:
input_dir = "/local_data/cosmiq/src/achadda/yolo_planes/yolov5/inference/class_four_out"
output_dir = input_dir
image_dir = "/local_data/cosmiq/src/achadda/yolo_planes/class_four/images/val"
lookup_table = "/local_data/cosmiq/src/achadda/yolo_planes/geojsons_test/yolo_class_four/custom_class_lookup.csv"
lookup_table = pd.read_csv(lookup_table)
lookup_table['yolo'] = lookup_table['custom_id'] - 1
lookup_table.set_index('yolo')
#lookup_table['num_tail_fins,wing_position,wing_type][0]
print(lookup_table)

     Unnamed: 0                             role  num_engines propulsion  \
0             0    Large Civil Transport/Utility            2        jet   
1             1    Large Civil Transport/Utility            2        jet   
2             2    Large Civil Transport/Utility            2        jet   
3             3    Large Civil Transport/Utility            2        jet   
4             4    Large Civil Transport/Utility            2        jet   
5             5    Large Civil Transport/Utility            3        jet   
6             6    Large Civil Transport/Utility            4        jet   
7             7    Large Civil Transport/Utility            4        jet   
8             8    Large Civil Transport/Utility            4        jet   
9             9    Large Civil Transport/Utility            4        jet   
10           10    Large Civil Transport/Utility            4  propeller   
11           11    Large Civil Transport/Utility            4  propeller   
12          

In [97]:
class_ids = []
classes = []
confidences = []
images = []
crs_s = []
bb_geoms = []
os.chdir(input_dir)
txts = glob.glob("*.txt")
for txt in tqdm(txts):
    image = txt.split(".")[0] + ".png"
    im_path = os.path.join(image_dir, image)
    image = image.split("_tile")[0]
    df = pd.read_csv(txt,header = None, sep = " ")
    for index, row in df.iterrows():
        b_box = box(row[0], row[1], row[2], row[3])
        bb_geom = convert_poly_coords(b_box, raster_src=im_path)
        bb_geoms.append(bb_geom)
        crs = get_crs(gdal.Open(im_path))
        crs = str(crs)
        crs_s.append(crs)
        class_ids.append(row[0])
        confidences.append(row[5])
        images.append(image)
        classes.append(str(lookup_table['role'][row[0]]) + ' ' + str(lookup_table['num_engines'][row[0]]) + ' ' + str(lookup_table['propulsion'][row[0]]) + ' ' + str(lookup_table['canards'][row[0]]) + ' ' + str(lookup_table['num_tail_fins'][row[0]]) + ' ' + str(lookup_table['wing_position'][row[0]]) + ' ' + str(lookup_table['wing_type'][row[0]]) + ' ' + str(lookup_table['faa_wingspan_class'][row[0]]))
        
dict = {'class_id': class_ids, 'class': classes, 'confidence': confidences, 'image': images, 'crs': crs_s, 'bb_geometry': bb_geoms}
df = pd.DataFrame(dict)





  0%|          | 0/2643 [00:00<?, ?it/s]


  0%|          | 7/2643 [00:00<00:40, 65.43it/s]


  1%|          | 15/2643 [00:00<00:38, 68.45it/s]


  1%|          | 19/2643 [00:00<01:06, 39.64it/s]


  1%|          | 23/2643 [00:00<01:16, 34.26it/s]


  1%|          | 33/2643 [00:00<01:01, 42.58it/s]


  2%|▏         | 43/2643 [00:00<00:50, 51.01it/s]


  2%|▏         | 55/2643 [00:00<00:42, 60.78it/s]


  3%|▎         | 67/2643 [00:00<00:36, 71.31it/s]


  3%|▎         | 78/2643 [00:01<00:32, 79.22it/s]


  3%|▎         | 88/2643 [00:01<00:31, 80.91it/s]


  4%|▎         | 98/2643 [00:01<00:30, 84.50it/s]


  4%|▍         | 110/2643 [00:01<00:28, 88.32it/s]


  5%|▍         | 120/2643 [00:01<00:27, 90.12it/s]


  5%|▌         | 133/2643 [00:01<00:25, 97.66it/s]


  5%|▌         | 144/2643 [00:01<00:28, 88.56it/s]


  6%|▌         | 154/2643 [00:01<00:30, 82.86it/s]


  6%|▌         | 164/2643 [00:02<00:28, 86.03it/s]


  7%|▋         | 178/2643 [00:02<00:25, 94.84it/s]


  7%|▋      

 49%|████▉     | 1291/2643 [00:15<00:15, 88.18it/s]


 49%|████▉     | 1303/2643 [00:16<00:14, 92.01it/s]


 50%|████▉     | 1314/2643 [00:16<00:14, 93.49it/s]


 50%|█████     | 1324/2643 [00:16<00:15, 83.12it/s]


 51%|█████     | 1336/2643 [00:16<00:14, 89.18it/s]


 51%|█████     | 1347/2643 [00:16<00:13, 93.88it/s]


 51%|█████▏    | 1357/2643 [00:16<00:15, 85.51it/s]


 52%|█████▏    | 1368/2643 [00:16<00:14, 89.75it/s]


 52%|█████▏    | 1380/2643 [00:16<00:13, 95.90it/s]


 53%|█████▎    | 1390/2643 [00:16<00:13, 93.09it/s]


 53%|█████▎    | 1403/2643 [00:17<00:12, 100.80it/s]


 53%|█████▎    | 1414/2643 [00:17<00:12, 96.40it/s] 


 54%|█████▍    | 1424/2643 [00:17<00:12, 96.59it/s]


 54%|█████▍    | 1434/2643 [00:17<00:13, 91.57it/s]


 55%|█████▍    | 1447/2643 [00:17<00:12, 97.04it/s]


 55%|█████▌    | 1457/2643 [00:17<00:12, 97.49it/s]


 56%|█████▌    | 1467/2643 [00:17<00:14, 83.54it/s]


 56%|█████▌    | 1476/2643 [00:17<00:14, 80.88it/s]


 56%|█████▌    | 1486/2643

 95%|█████████▌| 2522/2643 [00:31<00:01, 63.73it/s]


 96%|█████████▌| 2531/2643 [00:32<00:01, 68.98it/s]


 96%|█████████▌| 2540/2643 [00:32<00:01, 72.61it/s]


 97%|█████████▋| 2552/2643 [00:32<00:01, 81.99it/s]


 97%|█████████▋| 2561/2643 [00:32<00:01, 78.51it/s]


 97%|█████████▋| 2570/2643 [00:32<00:00, 79.15it/s]


 98%|█████████▊| 2579/2643 [00:32<00:00, 78.63it/s]


 98%|█████████▊| 2588/2643 [00:32<00:00, 72.27it/s]


 98%|█████████▊| 2596/2643 [00:32<00:00, 74.04it/s]


 99%|█████████▊| 2605/2643 [00:33<00:00, 77.85it/s]


 99%|█████████▉| 2614/2643 [00:33<00:00, 78.07it/s]


 99%|█████████▉| 2625/2643 [00:33<00:00, 83.26it/s]


100%|██████████| 2643/2643 [00:33<00:00, 78.95it/s]


In [98]:
df.to_csv(path_or_buf=os.path.join(output_dir, "detection_results.csv"))

In [99]:
vectorize_yolo(df, output_dir, nms=True, overlapThresh=0.5)

Executing non-max suppression...
  non-max suppression init boxes: 694
  non-max suppression final boxes: 197
Executing non-max suppression...
  non-max suppression init boxes: 247
  non-max suppression final boxes: 99
Executing non-max suppression...
  non-max suppression init boxes: 172
  non-max suppression final boxes: 33
Executing non-max suppression...
  non-max suppression init boxes: 123
  non-max suppression final boxes: 97
Executing non-max suppression...
  non-max suppression init boxes: 311
  non-max suppression final boxes: 222
Executing non-max suppression...
  non-max suppression init boxes: 141
  non-max suppression final boxes: 87
Executing non-max suppression...
  non-max suppression init boxes: 362
  non-max suppression final boxes: 270
Executing non-max suppression...
  non-max suppression init boxes: 37
  non-max suppression final boxes: 9
Executing non-max suppression...
  non-max suppression init boxes: 356
  non-max suppression final boxes: 202
Executing non-max

/opt/conda/envs/solaris/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/envs/solaris/lib/python3.6/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in greater


84
Executing non-max suppression...
  non-max suppression init boxes: 147
  non-max suppression final boxes: 64
Executing non-max suppression...
  non-max suppression init boxes: 64
  non-max suppression final boxes: 49
Executing non-max suppression...
  non-max suppression init boxes: 259
  non-max suppression final boxes: 165
Executing non-max suppression...
  non-max suppression init boxes: 143
  non-max suppression final boxes: 76
Executing non-max suppression...
  non-max suppression init boxes: 231
  non-max suppression final boxes: 121
Executing non-max suppression...
  non-max suppression init boxes: 119
  non-max suppression final boxes: 32
Executing non-max suppression...
  non-max suppression init boxes: 141
  non-max suppression final boxes: 98
Executing non-max suppression...
  non-max suppression init boxes: 144
  non-max suppression final boxes: 87
Executing non-max suppression...
  non-max suppression init boxes: 26
  non-max suppression final boxes: 11
Executing non-ma